In [ ]:
!pip install --quiet llama-index torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.7 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
import pandas as pd

nest_asyncio.apply()
from IPython.display import Markdown, display
import pandas as pd
from llama_index.core import Document

# Load the CSV file into a pandas DataFrame from the GitHub URL
news = pd.read_csv("https://raw.githubusercontent.com/jameshopham/Datasets/main/Combined_Clinical_Trials_Articles.csv")
#dataset is a combined database compiled in R with dplyr of news articles and research papers from 2000 to 2020 and contains the title of the paper as well as a summary of the respective document

#sample from database
sampled_df = news.sample(n=500)

#delete unused category column
sampled_df = sampled_df.iloc[:, :-1]

# Instantiate the Document objects with a text property
# Create a list of Document objects based on the DataFrame
documents = [Document(text=f"{row['Headlines']}: {row['Teaser']}") for _, row in sampled_df.iterrows()]
# Display the first few rows of the DataFrame to verify the contents
sampled_df.head()


,Date,Source,Headlines,Teaser
36452,"July, 2016",Medscape Medical News,'Weekend Effect' May Limit Use of Deceased Don...,"Weekend resource limitations, especially in sm..."
5538,June 2011,Future Microbiology,Therapeutic Options for Enterobacteriaceae Inf...,Carbapenem resistance in Enterobacteriaceae is...
26056,"December, 2014",Medscape Medical News,CT Contrast Agent Does Not Increase Risk for K...,A CT contrast agent was unrelated to nephropat...
4737,November 2018,Medscape Medical News,Repeated CBD Doses Required for Effective Pain...,Repeated administration of cannabidiol (CBD) i...
40177,"June, 2017",Medscape Medical News,Surgery May Help in Diabetic Neuropathy With S...,Nerve-decompression surgery in diabetic periph...


In [ ]:
%pip install --quiet llama-index-llms-mistralai llama-index-embeddings-mistralai
from llama_index.llms.mistralai import MistralAI
import time

api_key = "SV6IHBrxrmaOA3i6djxwZwYMKrlfi9YR"

start_time = time.time()
print("Initializing LLM...")
llm = MistralAI(api_key=api_key, model="mistral-medium")
print("LLM Initialized.")
end_time = time.time()
print(f"LLM Initialization took {end_time - start_time} seconds.")


Initializing LLM...
LLM Initialized.
LLM Initialization took 0.09852218627929688 seconds.


In [ ]:
from llama_index.embeddings.mistralai import MistralAIEmbedding
import time


start_time = time.time()
print("Initializing Embedding Model...")
embed_model = MistralAIEmbedding(model_name="mistral-embed", api_key=api_key)
print("Embedding Model Initialized.")
end_time = time.time()
print(f"Embedding Model Initialization took {end_time - start_time} seconds.")
from llama_index.core import Settings

# Configure global settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

Initializing Embedding Model...
Embedding Model Initialized.
Embedding Model Initialization took 0.11559844017028809 seconds.


In [ ]:
from typing import Literal
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

#Use schemllmpathextractor to define the types of nodes you want to extract
# best practice to use upper-case

#nodes
entities = Literal["DISEASE", "SYMPTOMS", "LOCATION", "ORGANIZATION", "RESEARCH", "TREATMENT", "CLINICAL TRIAL PHASE"]  #event is more ambiguous than the other typical extractions we listed
    #ambiguous labels allow the llm to extract a lot of information

#relationships to extract
relations = Literal[
    "DEVELOPING",
    "RESEARCHING",
    "PARTNERSHIP",
    "ACQUISITION",
    "DEVELOPED_BY",
    "APPROVED",
    "FUNDING",
    "TESTING",
    "TREATS",
    "CAUSES"
]

In [ ]:
# define which entities can have which relations
#not all relationships can be part of all node labels
validation_schema = {
    "DRUG": ["DEVELOPED_BY", "ORGANIZATION", "RESEARCHING", "TREATS"],
    "Organization": [
        "SUPPLIER_OF",
        "COMPETITOR",
        "PARTNERSHIP",
        "ACQUISITION"
        "RESEARCHING",
        "DEVELOPING",
        "TESTING",
        "FUNDING"
    ],
    "SYMPTOMS": ["CAUSES"], #'SYMPTOMS' only should have a relationship with 'CAUSES'
}

In [ ]:
from llama_index.core import PropertyGraphIndex

kg_extractor = SchemaLLMPathExtractor(
    llm=llm,
    possible_entities=entities, #pass the possible entities, relationship, validation schema to the llm
    possible_relations=relations,
    kg_validation_schema=validation_schema,
    # if false, allows for values outside of the schema
    # useful for using the schema as a suggestion
    strict=False, #strict mode: even if you provide instructions to the llm (eg which types of nodes or relationships it should use), it doesn't really mean the llm will follow them 100% correct
            #The strict mode was implemented bc since we know the types of relationships and nodes we expect, we can filter them out by setting strict=TRUE; if we want to leave any other nodes and relationships the LLM identified we set strict=FALSE
            #for models that are not good at following instructions (eg llms that are not native function calling models) we should filter strict=True
)

NUMBER_OF_ARTICLES = 125 #entire database contains 58067 articles/research papers
                    #too computationally heavy to run them all

index = PropertyGraphIndex.from_documents(
    documents[:NUMBER_OF_ARTICLES],
    kg_extractors=[kg_extractor],
    llm=llm,
    embed_model=embed_model,
    #property_graph_store=graph_store,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/125 [00:00<?, ?it/s]



Extracting paths from text with schema:   0%|          | 0/125 [00:00<?, ?it/s]

Extracting paths from text with schema:   1%|          | 1/125 [00:13<27:33, 13.34s/it]

Extracting paths from text with schema:   2%|▏         | 3/125 [00:14<07:35,  3.73s/it]

Extracting paths from text with schema:   3%|▎         | 4/125 [00:19<09:01,  4.47s/it]

Extracting paths from text with schema:   4%|▍         | 5/125 [00:26<10:36,  5.30s/it]

Extracting paths from text with schema:   5%|▍         | 6/125 [00:27<07:45,  3.91s/it]

Extracting paths from text with schema:   6%|▌         | 7/125 [00:30<07:06,  3.62s/it]

Extracting paths from text with schema:   6%|▋         | 8/125 [00:40<10:29,  5.38s/it]

Extracting paths from text with schema:   8%|▊         | 10/125 [00:43<07:10,  3.74s/it]

Extracting paths from text with schema:   9%|▉         | 11/125 [00:45<05:54,  3.11s/it]

Extracting paths from text with schema:  10%|▉         | 12/125 [00:54<09:09,  4.86s/it]

Extracting paths from te

In [ ]:
retriever = index.as_retriever(
    include_text=True,  # include source text, default True
)

nodes = retriever.retrieve("Can you please recommend me articles or paper topics to read that would be important for a professional specializing in cancer treatment to be aware of?")

for node in nodes:
    print(node.text)

Here are some facts extracted from the provided text:

Cancer patients with obesity -> HAVE_BETTER_RESPONSES_TO -> Immunotherapy

Rates of Eating Disorders Similar for US Boys, Girls Aged 9-10: Obesity increases the risk for cancer, but, somewhat paradoxically, cancer patients who are obese and who undergo immunotherapy have better responses.
Here are some facts extracted from the provided text:

Cancer patients with obesity -> HAVE_BETTER_RESPONSES_TO -> Immunotherapy
Cancer patients -> HAVE -> Obesity
Obesity -> INCREASES_RISK_FOR -> Cancer

Rates of Eating Disorders Similar for US Boys, Girls Aged 9-10: Obesity increases the risk for cancer, but, somewhat paradoxically, cancer patients who are obese and who undergo immunotherapy have better responses.


In [ ]:
query_engine = index.as_query_engine(
    include_text=True
)

response = query_engine.query("Can you please recommend me articles or paper topics to read that would be important for a professional specializing in cancer treatment to be aware of?")

display(Markdown(f"{response.response}"))

Certainly, here are some article and paper topics that could be important for a professional specializing in cancer treatment:

1. The impact of obesity on cancer risk and treatment outcomes: As the context information suggests, obesity increases the risk of cancer, but it may also have paradoxical effects on the response to immunotherapy in cancer patients. Understanding the complex relationship between obesity and cancer could be crucial for developing effective treatment strategies.
2. The role of immunotherapy in cancer treatment: Immunotherapy has emerged as a promising approach to cancer treatment, and it is important for cancer specialists to stay up-to-date on the latest developments in this field. Topics of interest might include the mechanisms of action of immunotherapy drugs, the selection of patients who are most likely to benefit from immunotherapy, and the management of immune-related adverse events.
3. Precision medicine in cancer treatment: Precision medicine involves tailoring cancer treatment to the individual patient based on their genetic profile and other factors. This approach has the potential to improve treatment outcomes and reduce side effects. Topics of interest might include the use of genomic testing to guide treatment decisions, the development of targeted therapies for specific genetic mutations, and the integration of precision medicine into clinical practice.
4. The impact of lifestyle factors on cancer risk and treatment outcomes: In addition to obesity, other lifestyle factors such as diet, exercise, and smoking can also affect cancer risk and treatment outcomes. Understanding the role of these factors could help cancer specialists provide more comprehensive care to their patients.
5. Disparities in cancer care: Disparities in cancer care can lead to unequal outcomes for patients based on factors such as race, ethnicity, and socioeconomic status. Addressing these disparities is an important challenge for cancer specialists, and topics of interest might include strategies for improving access to care, reducing disparities in treatment outcomes, and addressing social determinants of health.

In [ ]:
#customize further using a vector context retriever
'''
from llama_index.core.indices.property_graph import VectorContextRetriever, PGRetriever, LLMSynonymRetriever

vector_retriever = VectorContextRetriever(
    index.property_graph_store,
    # only needed when the graph store doesn't support vector queries
    # vector_store=index.vector_store,
    embed_model=embed_model,
    # include source chunk text with retrieved paths
    include_text=False,
    # the number of nodes to fetch
    similarity_top_k=2,
    # the depth of relations to follow after node retrieval
    path_depth=1,
    # can provide any other kwargs for the VectorStoreQuery class
)

retriever = index.as_retriever(sub_retrievers=[vector_retriever])

nodes = retriever.retrieve("<Can you please recommend me articles or paper topics to read that would be important for a professional specializing in cancer treatment to be aware of?>")
'''

In [ ]:
#example of query that is not represented on database sample
query_engine = index.as_query_engine(
    include_text=True
)

response = query_engine.query("What genetic mutations were increasingly targeted for research from 2010 to 2020?")

display(Markdown(f"{response.response}"))

The provided context does not contain any information about genetic mutations or research targeting them from 2010 to 2020. It only discusses the relationship between obesity, cancer, and immunotherapy, as well as the attendance at the ADA 72nd Scientific Sessions. Therefore, I cannot answer this query based on the given context.